# Watson Visual Insights



*By J Rogel-Salazar*, *Nov 2016*

# Getting Setup

Create an instance of visual insights in your bluemix [dashboard](https://console.ng.bluemix.net/) and save the associated credentials json to creds.json in the same directory as this IPython notebook.

In this case I am using Python 3.

GitHub [link](https://github.com/watson-developer-cloud/python-sdk) has the code to call the [Visual Watson API](https://github.com/watson-developer-cloud/python-sdk/blob/master/examples/visual_recognition_v3.py).

In [1]:
import os 

# You may need to change to the directoy where 
# your code lives. If so change the information below
# accordingly

# os.chdir('~/')
# print(os.getcwd())

In [2]:
import json

with open("cred_visual.json") as f:
    creds = json.load(f)['credentials']

# You may want to check that the credentials have
# been imported correctly:

# print("Username {api_key} \nGateway: {url}".format(**creds))

In [3]:
# Making sure that the zip files are in the expected location
if not os.path.exists("./data/positive.zip"):
    print("No positive examples found")
else:
    print("Positives found")

if not os.path.exists("./data/negative.zip"):
    print("No negative examples found")
else:
    print("Negatives found")

Positives found
Negatives found


# Call Visual Insights API

In [4]:
import json
from os.path import join, dirname
from os import environ
from watson_developer_cloud import VisualRecognitionV3

username = creds["api_key"]
psw = creds["url"]

visual_recognition = VisualRecognitionV3('2016-07-27', api_key=username)

print(visual_recognition)

Let us make sure that the Visual Insights API is working correctly. We will use a picture of Albert Einstein and see the information that Watson provides for it:

In [5]:
test_url = 'https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg'

![Einstein](https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg)

Here is what Watson tells us about the **classification** of this image: Watson tells us that this is a person... 

OK... 

In [6]:
print(json.dumps(visual_recognition.classify(images_url=test_url), indent=2))

{
  "custom_classes": 0,
  "images_processed": 1,
  "images": [
    {
      "resolved_url": "https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg",
      "classifiers": [
        {
          "classifier_id": "default",
          "classes": [
            {
              "class": "person",
              "type_hierarchy": "/people",
              "score": 1.0
            }
          ],
          "name": "default"
        }
      ],
      "source_url": "https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg"
    }
  ]
}


We can also see the infomration from the **face detection**. Watson tell us that is not a female, gives is a range for the age of the person an the location of the face in the picture. 

Not bad!

In [7]:
print(json.dumps(visual_recognition.detect_faces(images_url=test_url), indent=2))

{
  "images_processed": 1,
  "images": [
    {
      "resolved_url": "https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg",
      "source_url": "https://www.nobelprize.org/nobel_prizes/physics/laureates/1921/einstein.jpg",
      "faces": [
        {
          "gender": {
            "gender": "FEMALE",
            "score": 0.0
          },
          "face_location": {
            "left": 50,
            "width": 88,
            "height": 138,
            "top": 53
          },
          "age": {
            "max": 54,
            "min": 45,
            "score": 0.348709
          }
        }
      ]
    }
  ]
}


# Distinguishing Cats from Dogs (or actually other "animals")


We will use the Watson Visual Insights API to create a classifier that tells us whether the animal in a picture is a cat or not. 

In this case we are training the classifier with positive examples (cats) versus negative ones (dogs). 

We use Python to send the requests to the Watson API and use the `create_classifier` method to create a classifier calls 'Dogs vs Cats' with images in the zip files called 'negative.zip' and 'positive.zip'


In [10]:
with open('./data/negative.zip', 'rb') as dogs, open('./data/positive.zip', 'rb') as cats:
    my_classifier = visual_recognition.create_classifier('Dogs vs Cats',
                                                         cats_positive_examples=cats,
                                                         negative_examples=dogs)
    print(json.dumps(my_classifier, indent=2))

{
  "name": "Dogs vs Cats",
  "status": "training",
  "classifier_id": "DogsvsCats_751461720",
  "owner": "d671fed5-b775-4e9f-8f3a-29d90aae8d78",
  "created": "2016-11-17T14:10:21.041Z",
  "classes": [
    {
      "class": "cats"
    }
  ]
}


Let us see the metadata for the custom classifier we just created. The identifier of the classifier is the 'classifier_id'. 

In [20]:
custom_clf = visual_recognition.list_classifiers()

clf_id = custom_clf['classifiers'][0]['classifier_id']

print(clf_id)

DogsvsCats_751461720


In the plan used for this example we can only have one single custom classifier. If you need a new one, you will need to delete the existing one. This can be done with the `delete_classifier` method. Only run the following line if you need to delete the classfier. 

In [9]:

# deleting a classifier
print(json.dumps(visual_recognition.delete_classifier(classifier_id=clf_id),
                 indent=2))

{}


# Telling cats from dogs

Let us now use the classifier to tell cats from dogs in test images (not seen by the classifier during training).

In this case we have some images in a file called "mypets.zip"

In [21]:
with open('./data/mypets.zip', 'rb') as image_file:
    mypets_cf = visual_recognition.classify(images_file=image_file,\
                                                  classifier_ids=[clf_id])
    print(json.dumps(mypets_cf, indent=2))

{
  "custom_classes": 1,
  "images_processed": 7,
  "images": [
    {
      "classifiers": [],
      "image": "./data/mypets.zip/five.jpg"
    },
    {
      "classifiers": [
        {
          "classifier_id": "DogsvsCats_751461720",
          "classes": [
            {
              "class": "cats",
              "score": 0.554423
            }
          ],
          "name": "Dogs vs Cats"
        }
      ],
      "image": "./data/mypets.zip/four.jpg"
    },
    {
      "classifiers": [
        {
          "classifier_id": "DogsvsCats_751461720",
          "classes": [
            {
              "class": "cats",
              "score": 0.573919
            }
          ],
          "name": "Dogs vs Cats"
        }
      ],
      "image": "./data/mypets.zip/one.jpg"
    },
    {
      "classifiers": [],
      "image": "./data/mypets.zip/seven.jpg"
    },
    {
      "classifiers": [],
      "image": "./data/mypets.zip/six.jpg"
    },
    {
      "classifiers": [],
      "image": "./da

In [22]:
# To classify a single image:

with open('./data/one.jpg', 'rb') as image_file:
    print(json.dumps(visual_recognition.classify(images_file=image_file, 
                                                 classifier_ids=[clf_id]), 
                     indent=2))

with open('./data/six.jpg', 'rb') as image_file:
    print(json.dumps(visual_recognition.classify(images_file=image_file, 
                                                 classifier_ids=[clf_id]), 
                     indent=2))

{
  "custom_classes": 1,
  "images_processed": 1,
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "DogsvsCats_751461720",
          "classes": [
            {
              "class": "cats",
              "score": 0.573919
            }
          ],
          "name": "Dogs vs Cats"
        }
      ],
      "image": "./data/one.jpg"
    }
  ]
}
{
  "custom_classes": 1,
  "images_processed": 1,
  "images": [
    {
      "classifiers": [],
      "image": "./data/six.jpg"
    }
  ]
}


As we can see the information returned by the Watson API is a nested dictionary with json-ified fields. We can use some functions to get information from the dictionary. 

Let us build a function that allows us to get the image id and name and provide the score returned by the Watson API classifier and the class (cats or dogs in this case) that the image belongs to. We write the output to a comma-separeted-value file.

In [23]:
def ClassifierInfo(Clf, FileName='./data/Classifier.txt', Alt_class='0', sep='\t',
                   verbose=False):

    import fileinput
    
    num_images = Clf['images_processed']
    savefl = open(FileName, 'w+')
    
    myline = ['Image_Id', 'Image_Name', 'Score', "Class"]
    mystr = sep.join(myline) +'\n'
    savefl.write(mystr)

    for i in range(0,num_images):
        id = i

        imageName = Clf['images'][i]['image']
        print("Processing " + imageName)
        classifier = Clf['images'][i]['classifiers']

        if len(classifier)!=0:
            my_score = classifier[0]['classes'][0]['score']
            my_class = classifier[0]['classes'][0]['class']

        else:
            my_score = '1'
            my_class = Alt_class

        myline = [str(id), imageName, str(my_score), my_class]
        mystr = sep.join(myline) +'\n'
        if verbose:
            print(mystr)
        savefl.write(mystr)
    
    savefl.truncate()
    savefl.close()

In [24]:
ClassifierInfo(mypets_cf, Alt_class='dogs', sep=',', verbose=True)

Processing ./data/mypets.zip/five.jpg
0,./data/mypets.zip/five.jpg,1,dogs

Processing ./data/mypets.zip/four.jpg
1,./data/mypets.zip/four.jpg,0.554423,cats

Processing ./data/mypets.zip/one.jpg
2,./data/mypets.zip/one.jpg,0.573919,cats

Processing ./data/mypets.zip/seven.jpg
3,./data/mypets.zip/seven.jpg,1,dogs

Processing ./data/mypets.zip/six.jpg
4,./data/mypets.zip/six.jpg,1,dogs

Processing ./data/mypets.zip/three.jpg
5,./data/mypets.zip/three.jpg,1,dogs

Processing ./data/mypets.zip/two.jpg
6,./data/mypets.zip/two.jpg,0.541283,cats



Et voilà! 

Enjoy telling cats from dogs in the many images available in the interwebs.